# Capstone Project: Feature Selection and Engineering 

### Imports

In [1]:
import pandas as pd
import numpy as np
import geopy.distance
import numbers
from functools import reduce

### Dataset

In [2]:
capstone_df = pd.read_csv("../data/location_with_google_ratings.csv")

In [3]:
capstone_df.head()

,area,county,city_or_town,lat_area,long_area,population,median_household_income,income_per_capita,name,address,lat_venue,long_venue,categories,price_level,rating,user_ratings_total
0,Alameda,Alameda,City,37.734379,-122.23952,77624,104756,52448,Chuck Corica Golf Complex,1 Clubhouse Memorial Rd,37.738885,-122.234326,Golf Course,0.0,0.0,0.0
1,Alameda,Alameda,City,37.734379,-122.23952,77624,104756,52448,Shoreline Park,"Alameda, CA",37.742580,-122.250126,Park,0.0,0.0,0.0
2,Alameda,Alameda,City,37.734379,-122.23952,77624,104756,52448,Angel Fish Japanese Cuisine,883 C-2 Island Dr,37.737478,-122.240107,Japanese Restaurant,2.0,4.3,112.0
3,Alameda,Alameda,City,37.734379,-122.23952,77624,104756,52448,La Penca Azul,891 Island Dr Ste B (at Mecartney Rd),37.737667,-122.240767,Mexican Restaurant,2.0,4.3,2263.0
4,Alameda,Alameda,City,37.734379,-122.23952,77624,104756,52448,La Val's Pizza,891 Island Dr,37.737610,-122.241001,Italian Restaurant,2.0,4.4,190.0


In [4]:
capstone_df = capstone_df[capstone_df["area"] == "San Francisco"]
capstone_df.reset_index(inplace = True)
capstone_df.shape

(1450, 17)

### Population and Density Metrics

#### User Rating as % of Population

In [5]:
user_as_pct_pop = capstone_df[["name","categories","area","lat_venue","long_venue","population","user_ratings_total"]]

In [6]:
user_as_pct_pop["user_rate_as_pct_pop"] = user_as_pct_pop["user_ratings_total"] / user_as_pct_pop["population"]

<ipython-input-6-a4965d6fb73e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_as_pct_pop["user_rate_as_pct_pop"] = user_as_pct_pop["user_ratings_total"] / user_as_pct_pop["population"]


In [7]:
user_as_pct_pop.head()

,name,categories,area,lat_venue,long_venue,population,user_ratings_total,user_rate_as_pct_pop
0,Sunset Recreation Center,Playground,San Francisco,37.757310,-122.487072,881549,0.0,0.000000
1,Terra Cotta Warrior,Chinese Restaurant,San Francisco,37.761118,-122.489612,881549,0.0,0.000000
2,Yuanbao Jiaozi,Dumpling Restaurant,San Francisco,37.763494,-122.480616,881549,287.0,0.000326
3,Thai Nghiep Ky Mi Gia,Vietnamese Restaurant,San Francisco,37.754045,-122.479119,881549,184.0,0.000209
4,Home,Coffee Shop,San Francisco,37.754348,-122.476973,881549,478.0,0.000542


In [8]:
user_as_pct_pop = pd.DataFrame(user_as_pct_pop.groupby(["area","categories"])["user_rate_as_pct_pop"].mean())

In [9]:
user_as_pct_pop.head()

user_rate_as_pct_pop
area          categories                              
San Francisco Acai House                      0.000000
              Accessories Store               0.000049
              African Restaurant              0.000192
              Airport                         0.000000
              Airport Lounge                  0.001183

#### Venue and Coffee Shop per Capita

In [10]:
per_cap = capstone_df[["area","name","lat_venue","long_venue","population","categories"]]

In [11]:
per_cap["id"] = per_cap["name"]+str(per_cap["lat_venue"])+str(per_cap["long_venue"])

<ipython-input-11-5750b99af896>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_cap["id"] = per_cap["name"]+str(per_cap["lat_venue"])+str(per_cap["long_venue"])


In [12]:
ven_per_cap = pd.DataFrame(per_cap.groupby(["area","population"])["id"].count())

In [13]:
ven_per_cap["venues_per_capita"] = ven_per_cap["id"] / ven_per_cap.index.get_level_values("population")

In [14]:
print(ven_per_cap.shape)
ven_per_cap.head()

(1, 2)


,,id,venues_per_capita
area,population,,
San Francisco,881549,1450,0.001645


In [15]:
cs_per_cap = pd.DataFrame(per_cap[per_cap["categories"] == "Coffee Shop"].groupby(["area","population"])["id"].count())

In [16]:
cs_per_cap["cs_per_capita"] = cs_per_cap["id"] / cs_per_cap.index.get_level_values("population")

In [17]:
print(cs_per_cap.shape)
cs_per_cap.head()

(1, 2)


,,id,cs_per_capita
area,population,,
San Francisco,881549,69,0.000078


### Distance Matrix

In [18]:
# Thanks Gwen for the source!
# Source: https://gis.stackexchange.com/questions/293310/how-to-use-geoseries-distance-to-get-the-right-answer/346064#346064

In [19]:
distance_df = capstone_df[["area","name","lat_venue","long_venue","categories"]]

In [20]:
distance_df["id"] = distance_df["name"] + str(distance_df["lat_venue"]) + str(distance_df["long_venue"])

<ipython-input-20-df378a32db13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance_df["id"] = distance_df["name"] + str(distance_df["lat_venue"]) + str(distance_df["long_venue"])


#### Venue per Area

In [21]:
ven_per_area = pd.DataFrame(distance_df.groupby(["area","name"])[["lat_venue","long_venue"]].mean())

In [22]:
ven_per_area.head()

lat_venue  long_venue
area          name                                            
San Francisco 1512 Barber Shop           37.789612 -122.420865
              22nd & Irving Market       37.763323 -122.480479
              22nd Street Jungle Stairs  37.754382 -122.436144
              24th & Mission Taco House  37.616197 -122.389738
              440 Castro                 37.761852 -122.435195

In [23]:
area_dic = {}

for area in ven_per_area.index.get_level_values("area").unique():
    area_dic[area] = [venue for venue in ven_per_area[ven_per_area.index.get_level_values("area") == area].index.get_level_values("name").unique()]
        

In [24]:
for k,v in area_dic.items():
    for venue in v:
        print(venue)
        for ven in ven_per_area.T.columns:
            lat_1 = ven_per_area[(ven_per_area.index.get_level_values("area") == k) &
                 (ven_per_area.index.get_level_values("name") == venue)]["lat_venue"].values
            long_1 = ven_per_area[(ven_per_area.index.get_level_values("area") == k) &
                 (ven_per_area.index.get_level_values("name") == venue)]["long_venue"].values
            lat_2 = ven_per_area[(ven_per_area.index.get_level_values("area") == ven[0]) &
                 (ven_per_area.index.get_level_values("name") == ven[1])]["lat_venue"].values
            long_2 = ven_per_area[(ven_per_area.index.get_level_values("area") == ven[0]) &
                 (ven_per_area.index.get_level_values("name") == ven[1])]["long_venue"].values
            distance = geopy.distance.geodesic((lat_1,long_1),(lat_2,long_2))
            ven_per_area.loc[(k,venue),ven[1]] = distance.meters

1512 Barber Shop
22nd & Irving Market
22nd Street Jungle Stairs
24th & Mission Taco House
440 Castro
7 Mile House
85C Bakery Cafe - Stonestown
9:20 Special
A16
AJ's Barbeque Cafe
AK Subs
ARCH Art and Drafting Supply
Absinthe Brasserie & Bar
Ace Wasabi's Rock-N-Roll Sushi
Acme Bread Company
Acquerello
AcroSports
Adam's Grub Truck
Adobe Books & Art Cooperative
Akiko’s Restaurant & Sushi Bar
Akna
Al's Attire
Al's Place
Alemany Farm
Alemany Farmers Market
Alexander Book Company
Alimento
All Good Pizza
Allbirds
Alley Cat Books
Alta Plaza Park
Ambassador Toys
Amelie
Amoeba San Francisco
Ananda Fuara
Anchor Brewing Company
Anchor Oyster Bar
Anchor Public Taps
Andersen Bread
Andytown
Andytown Roastery, Training Lab & Coffee Supply
Angelo J. Rossi Baseball Field
Angler
Anjalee Thai Massage
Anthony's Cookies
Apple Chestnut Street
Apple Stonestown
Apple Union Square
Aracely Cafe
Arbor
Archimedes Banya
Arizmendi Bakery
Arizmendi Bakery Panaderia & Pizzeria
Arlequin Wine Merchant
Aroma Tea Shop 茗茶轩

Humphry Slocombe
Huntington Falls
Ike's Place
Il Pollaio
Immigrant Point Overlook
Indian Paradox
Inn at the Presidio
Inner Sunset Farmers Market
Inspiration Point
Intelligentsia Roasting Works
InterContinental Mark Hopkins
International Orange
Iron and Resin SF
Irving Subs
Isa Restaurant
Iyara Traditional Thai Massage
Izakaya Sozai
J Georgie's Donut & Hamburgers
JJ Fish & Chicken
JOE & THE JUICE
Jack Early Park
Jackson Park & Playground
Jackson Place Cafe
Jackson Square
Jamba Juice
Jane
Jane the Bakery
Jang Soo BBQ
Japanese Toys
Jeffrey's Natural Pet Foods
Jiangnan Cuisine
Jimmy John's
Joe DiMaggio Playground
John McLaren Dog Run
John McLaren Park
John McLaren Park Lookout Point
Juice Shop
Julius Kahn Playground & Clubhouse
Just Won Ton
Kabuki Springs & Spa
Kamei Restaurant Supply
Karaweik Burmese Cuisine
Kasa Indian Eatery
Kezar Stadium
Kezar Triangle
King of Noodles
Kingdom of Dumpling
Kinjo
Kinokuniya Bookstore
Kirby Cove
Kitchen Istanbul
Kite Hill Open Space
Klein's Deli & Coffee B

See's Candies
Seniore's Pizza
Serenity MedSpa
Sessions at the Presidio
Seven Hills
Seven Stills
Sextant Coffee Roasters
Señor Sisig
Shabu House
Sheng Kee Bakery
Shinola
Shish Ke Baba
Siam Orchid Traditional Thai Massage
Sidewalk Juice
Sightglass Coffee
Sigmund Stern Grove
Silver Star Deli
Silver Terrace Soccer Field
Simple Pleasures Cafe
Slanted Door
Slice House by Tony Gemignani
Slim's
Sloat Garden Center
Smitten Ice Cream
Snowbird Coffee
SoMa StrEat Food Park
Sockshop Haight Street
Socola Chocolatier + Barista
Sol Food Puerto Rican Cuisine
Soma Eats
Song Tea & Ceramics
Soothe
Sotto Mare Oysteria and Seafood Restaurant
SoulCycle SoMa
Souvla
Speakeasy Ales & Lagers
Spice Ace
Spike's Coffee & Tea
Sports Basement
Spro Coffeelab
St. Francis Wood Fountain
St. Francis Wood Park
St. Mary's Park Dog Run
St. Mary's Park Playground
Stagecoach Greens
Starbelly
Starbucks
Starbucks Reserve
State Bird Provisions
Stella Pastry and Cafe
Stern Grove Dog Park
Stinson Beach
Stonestown Farmers Market
Sto

In [25]:
ven_per_area.head()

lat_venue  long_venue  \
area          name                                               
San Francisco 1512 Barber Shop           37.789612 -122.420865   
              22nd & Irving Market       37.763323 -122.480479   
              22nd Street Jungle Stairs  37.754382 -122.436144   
              24th & Mission Taco House  37.616197 -122.389738   
              440 Castro                 37.761852 -122.435195   

                                         1512 Barber Shop  \
area          name                                          
San Francisco 1512 Barber Shop                   0.000000   
              22nd & Irving Market            6007.945911   
              22nd Street Jungle Stairs       4135.468957   
              24th & Mission Taco House      19442.160376   
              440 Castro                      3329.716011   

                                         22nd & Irving Market  \
area          name                                              
San Francisco 1512 Barber Shop                    6007.945911   
              22nd & Irving Market                   0.000000   
              22nd Street Jungle Stairs           4030.833073   
              24th & Mission Taco House          18185.456617   
              440 Castro                          3993.519828   

                                         22nd Street Jungle Stairs  \
area          name                                                   
San Francisco 1512 Barber Shop                         4135.468957   
              22nd & Irving Market                     4030.833073   
              22nd Street Jungle Stairs                   0.000000   
              24th & Mission Taco House               15873.982672   
              440 Castro                                833.343717   

                                         24th & Mission Taco House  \
area          name                                                   
San Francisco 1512 Barber Shop                        19442.160376   
              22nd & Irving Market                    18185.456617   
              22nd Street Jungle Stairs               15873.982672   
              24th & Mission Taco House                   0.000000   
              440 Castro                              16656.049791   

                                           440 Castro  7 Mile House  \
area          name                                                    
San Francisco 1512 Barber Shop            3329.716011   9538.214741   
              22nd & Irving Market        3993.519828   9183.160555   
              22nd Street Jungle Stairs    833.343717   6104.026944   
              24th & Mission Taco House  16656.049791   9906.716065   
              440 Castro                     0.000000   6836.626476   

                                         85C Bakery Cafe - Stonestown  \
area          name                                                      
San Francisco 1512 Barber Shop                            8410.344628   
              22nd & Irving Market                        3947.025298   
              22nd Street Jungle Stairs                   4598.135742   
              24th & Mission Taco House                  14562.040265   
              440 Castro                                  5224.804356   

                                         9:20 Special  ...         Zazie  \
area          name                                     ...                 
San Francisco 1512 Barber Shop            1833.706888  ...   3720.743142   
              22nd & Irving Market        4250.885785  ...   2701.407255   
              22nd Street Jungle Stairs   3457.544760  ...   1713.816842   
              24th & Mission Taco House  19304.501812  ...  17377.489485   
              440 Castro                  2651.380332  ...   1352.145828   

                                         Zen Beauty Spa  b. on the go  \
area          name                                                      
San Francisco 1512 Barb

#### Average Distance per Category

In [52]:
cat_per_area = pd.merge(ven_per_area,
                        distance_df,
                        on = ["lat_venue","long_venue"],
                        how = "left"
                       )

In [53]:
cat_df = distance_df[["name","categories","id"]].drop_duplicates()

cat_dict = {}
for i in cat_df["id"].unique():
    cat_dict[cat_df[cat_df["id"] == i]["name"].values[0]] = cat_df[cat_df["id"] == i]["categories"].values[0]

In [54]:
cat_per_area.rename(columns=cat_dict,inplace = True)

In [55]:
cat_per_area.head()

,lat_venue,long_venue,Salon / Barbershop,Grocery Store,Trail,Taco Place,Gay Bar,Bar,Bakery,Nightclub,...,Bubble Tea Shop,Pizza Place,Wine Bar,Clothing Store,Bar,Salad Place,area,name,categories,id
0,37.789612,-122.420865,0.000000,6007.945911,4135.468957,19442.160376,3329.716011,9538.214741,8410.344628,1833.706888,...,6061.289800,1535.702898,3699.675243,1329.588153,662.617570,1975.442154,San Francisco,1512 Barber Shop,Salon / Barbershop,1512 Barber Shop0 37.757310\n1 37....
1,37.763323,-122.480479,6007.945911,0.000000,4030.833073,18185.456617,3993.519828,9183.160555,3947.025298,4250.885785,...,66.437555,7522.542692,2681.904927,7223.497569,6294.847015,7684.420389,San Francisco,22nd & Irving Market,Grocery Store,22nd & Irving Market0 37.757310\n1 ...
2,37.754382,-122.436144,4135.468957,4030.833073,0.000000,15873.982672,833.343717,6104.026944,4598.135742,3457.544760,...,4096.845914,5505.672824,1768.349558,4788.846331,4743.433717,4923.451428,San Francisco,22nd Street Jungle Stairs,Trail,22nd Street Jungle Stairs0 37.757310\n1 ...
3,37.616197,-122.389738,19442.160376,18185.456617,15873.982672,0.000000,16656.049791,9906.716065,14562.040265,19304.501812,...,18222.540547,20288.251913,17435.066127,19361.331017,20104.414541,19002.875224,San Francisco,24th & Mission Taco House,Taco Place,24th & Mission Taco House0 37.757310\n1 ...
4,37.761852,-122.435195,3329.716011,3993.519828,833.343717,16656.049791,0.000000,6836.626476,5224.804356,2651.380332,...,4059.670017,4740.574523,1391.025942,4073.358309,3924.880522,4281.260698,San Francisco,440 Castro,Gay Bar,440 Castro0 37.757310\n1 37.761118...


In [56]:
cat_per_area.drop(columns=["lat_venue","long_venue"],inplace = True)
cat_per_area.drop(columns = ["id"], inplace = True)

In [57]:
cat_per_area.head()

,Salon / Barbershop,Grocery Store,Trail,Taco Place,Gay Bar,Bar,Bakery,Nightclub,Italian Restaurant,BBQ Joint,...,Vietnamese Restaurant,Bubble Tea Shop,Pizza Place,Wine Bar,Clothing Store,Bar,Salad Place,area,name,categories
0,0.000000,6007.945911,4135.468957,19442.160376,3329.716011,9538.214741,8410.344628,1833.706888,2197.812090,7963.730255,...,2261.012138,6061.289800,1535.702898,3699.675243,1329.588153,662.617570,1975.442154,San Francisco,1512 Barber Shop,Salon / Barbershop
1,6007.945911,0.000000,4030.833073,18185.456617,3993.519828,9183.160555,3947.025298,4250.885785,5299.903510,5604.160091,...,7914.473101,66.437555,7522.542692,2681.904927,7223.497569,6294.847015,7684.420389,San Francisco,22nd & Irving Market,Grocery Store
2,4135.468957,4030.833073,0.000000,15873.982672,833.343717,6104.026944,4598.135742,3457.544760,5097.458248,3837.514074,...,5049.621749,4096.845914,5505.672824,1768.349558,4788.846331,4743.433717,4923.451428,San Francisco,22nd Street Jungle Stairs,Trail
3,19442.160376,18185.456617,15873.982672,0.000000,16656.049791,9906.716065,14562.040265,19304.501812,20922.880568,12631.839063,...,18904.715686,18222.540547,20288.251913,17435.066127,19361.331017,20104.414541,19002.875224,San Francisco,24th & Mission Taco House,Taco Place
4,3329.716011,3993.519828,833.343717,16656.049791,0.000000,6836.626476,5224.804356,2651.380332,4284.492572,4664.669763,...,4437.166413,4059.670017,4740.574523,1391.025942,4073.358309,3924.880522,4281.260698,San Francisco,440 Castro,Gay Bar


In [58]:
cat_per_area = pd.DataFrame(cat_per_area.groupby(["area","categories","name"]).mean())

In [59]:
cat_per_area_t = cat_per_area.T

In [60]:
# Source: https://stackoverflow.com/questions/40311987/pandas-mean-of-columns-with-the-same-names

cat_per_area_t = cat_per_area_t.groupby(by=cat_per_area_t.index, axis = 0).apply(lambda g: g.mean() if isinstance(g.iloc[0,0], numbers.Number) else g.iloc[0])

In [92]:
cat_per_area = cat_per_area_t.T

In [119]:
cat_per_area.head()

,categories,name,Acai House,Accessories Store,African Restaurant,Airport,Airport Lounge,Airport Service,Alternative Healer,American Restaurant,...,Video Game Store,Vietnamese Restaurant,Wagashi Place,Waterfall,Whisky Bar,Windmill,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
area,,,,,,,,,,,,,,,,,,,,,
San Francisco,Acai House,Blue Hawaii Açaí Café,0.000000,12390.638346,4409.203400,19769.078464,19864.970497,19637.364103,3865.175985,4471.327568,...,3751.590132,5681.672218,807.346888,7267.561832,607.292578,10185.706630,4406.601927,4316.937788,1776.379198,6409.645027
San Francisco,Accessories Store,Coach,19657.561753,8921.409814,16500.499815,350.278459,330.409949,417.205791,20661.813705,17750.797057,...,22900.119990,16957.926636,18984.613310,18303.115343,19362.523886,19918.205433,17711.354606,17670.738496,19783.919906,15843.898038
San Francisco,Accessories Store,Goorin Bros. Hat Shop,5044.150569,8878.924303,4913.688129,17810.483593,17868.986719,17703.968439,3270.820319,4868.367787,...,8568.152677,4429.060198,4335.713513,2432.496356,4436.943069,5572.933070,3931.412158,3207.879982,3666.299697,4410.478314
San Francisco,Accessories Store,Sockshop Haight Street,5446.588832,8908.429574,5223.794971,17872.207917,17926.844217,17768.511901,3422.825086,5135.743552,...,8930.532638,4474.697670,4750.140937,2001.000164,4839.378370,5145.400207,4127.604626,3354.806134,4017.493851,4494.291723
San Francisco,Accessories Store,Sunglass Hut,19414.252232,8843.519328,16269.925168,355.700031,478.638729,250.411778,20478.975122,17588.877509,...,22630.292627,16780.084905,18748.523592,18190.422836,19127.670133,19859.120074,17523.157290,17484.267358,19569.051677,15672.623411


In [120]:
cat_per_area.reset_index(inplace = True)
cat_per_area.set_index(["area","categories"], inplace = True)

### Category Frequency

In [121]:
# Inspiration: https://towardsdatascience.com/store-locations-d1025df22865

In [122]:
freq_cat = capstone_df[["area","categories","name","lat_venue","long_venue"]]

In [123]:
freq_cat["id"] = freq_cat["name"] + str(freq_cat["lat_venue"]) + str(freq_cat["long_venue"])

<ipython-input-123-ec97bec77cb6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  freq_cat["id"] = freq_cat["name"] + str(freq_cat["lat_venue"]) + str(freq_cat["long_venue"])


In [124]:
cat_total = pd.DataFrame(freq_cat.groupby(["area","categories"])["id"].count())

In [125]:
cat_total.head()

id
area          categories            
San Francisco Acai House           1
              Accessories Store    4
              African Restaurant   2
              Airport              1
              Airport Lounge       7

In [126]:
cat_total["freq"] = cat_total["id"] / cat_total["id"].sum()

In [127]:
cat_total.drop(columns = "id", inplace = True)

### Final Dataset

In [128]:
merged_df = pd.merge(cat_per_area,
                    user_as_pct_pop,
                    on = ["area","categories"],
                    how = "left"
                   )


final_df = pd.merge(merged_df,
                    cat_total,
                    on = ["area","categories"],
                    how = "left"
                   )

In [129]:
final_df.head()

name    Acai House  \
area          categories                                                
San Francisco Acai House          Blue Hawaii Açaí Café      0.000000   
              Accessories Store                   Coach  19657.561753   
              Accessories Store   Goorin Bros. Hat Shop   5044.150569   
              Accessories Store  Sockshop Haight Street   5446.588832   
              Accessories Store            Sunglass Hut  19414.252232   

                                 Accessories Store  African Restaurant  \
area          categories                                                 
San Francisco Acai House              12390.638346         4409.203400   
              Accessories Store        8921.409814        16500.499815   
              Accessories Store        8878.924303         4913.688129   
              Accessories Store        8908.429574         5223.794971   
              Accessories Store        8843.519328        16269.925168   

                                      Airport  Airport Lounge  \
area          categories                                        
San Francisco Acai House         19769.078464    19864.970497   
              Accessories Store    350.278459      330.409949   
              Accessories Store  17810.483593    17868.986719   
              Accessories Store  17872.207917    17926.844217   
              Accessories Store    355.700031      478.638729   

                                 Airport Service  Alternative Healer  \
area          categories                                               
San Francisco Acai House            19637.364103         3865.175985   
              Accessories Store       417.205791        20661.813705   
              Accessories Store     17703.968439         3270.820319   
              Accessories Store     17768.511901         3422.825086   
              Accessories Store       250.411778        20478.975122   

                                 American Restaurant  Antique Shop  ...  \
area          categories                                            ...   
San Francisco Acai House                 4471.327568   5998.301290  ...   
              Accessories Store         17750.797057  18066.255730  ...   
              Accessories Store          4868.367787   4196.255404  ...   
              Accessories Store          5135.743552   4198.856441  ...   
              Accessories Store         17588.877509  17903.203556  ...   

                                 Wagashi Place     Waterfall    Whisky Bar  \
area          categories                                                     
San Francisco Acai House            807.346888   7267.561832    607.292578   
              Accessories Store   18984.613310  18303.115343  19362.523886   
              Accessories Store    4335.713513   2432.496356   4436.943069   
              Accessories Store    4750.140937   2001.000164   4839.378370   
              Accessories Store   18748.523592  18190.422836  19127.670133   

                                     Windmill      Wine Bar     Wine Shop  \
area          categories                                                    
San Francisco Acai House         10185.706630   4406.601927   4316.937788   
              Accessories Store  19918.205433  17711.354606  17670.738496   
              Accessories Store   5572.933070   3931.412158   3207.879982   
              Accessories Store   5145.400207   4127.604626   3354.806134   
              Accessories Store  19859.120074  17523.157290  17484.267358   

                                  Wings Joint   Yoga Studio  \
area          categories                                      
San Francisco Acai House          1776.379198   6409.645027   
              Accessories Store  19783.919906  15843.898038   
              Accessories Store   3666.299697   4410.478314   
              Accessories Store   4017.493851   4494.291723   
              Accessories Store  19569.051677  15672.623411   

    

In [130]:
final_df["venues_per_capita"] = 0

for index in final_df.index:
    final_df.loc[index, "venues_per_capita"] = ven_per_cap[ven_per_cap.index.get_level_values("area") == index[0]]["venues_per_capita"].values[0]

In [131]:
final_df["cs_per_capita"] = 0

for index in final_df.index:
    if index[0] in cs_per_cap.index.get_level_values("area").unique():
        final_df.loc[index, "cs_per_capita"] = cs_per_cap[cs_per_cap.index.get_level_values("area") == index[0]]["cs_per_capita"].values[0]
    else:
        0

In [132]:
final_df = pd.merge(final_df,
                    capstone_df[["area","categories","name","price_level","rating","user_ratings_total"]],
                    on = ["area","categories","name"],
                    how = "left"
                   )

In [133]:
final_df.set_index(["area","categories","name"], inplace = True)
final_df.head()

Acai House  \
area          categories        name                                   
San Francisco Acai House        Blue Hawaii Açaí Café       0.000000   
              Accessories Store Coach                   19657.561753   
                                Goorin Bros. Hat Shop    5044.150569   
                                Sockshop Haight Street   5446.588832   
                                Sunglass Hut            19414.252232   

                                                        Accessories Store  \
area          categories        name                                        
San Francisco Acai House        Blue Hawaii Açaí Café        12390.638346   
              Accessories Store Coach                         8921.409814   
                                Goorin Bros. Hat Shop         8878.924303   
                                Sockshop Haight Street        8908.429574   
                                Sunglass Hut                  8843.519328   

                                                        African Restaurant  \
area          categories        name                                         
San Francisco Acai House        Blue Hawaii Açaí Café          4409.203400   
              Accessories Store Coach                         16500.499815   
                                Goorin Bros. Hat Shop          4913.688129   
                                Sockshop Haight Street         5223.794971   
                                Sunglass Hut                  16269.925168   

                                                             Airport  \
area          categories        name                                   
San Francisco Acai House        Blue Hawaii Açaí Café   19769.078464   
              Accessories Store Coach                     350.278459   
                                Goorin Bros. Hat Shop   17810.483593   
                                Sockshop Haight Street  17872.207917   
                                Sunglass Hut              355.700031   

                                                        Airport Lounge  \
area          categories        name                                     
San Francisco Acai House        Blue Hawaii Açaí Café     19864.970497   
              Accessories Store Coach                       330.409949   
                                Goorin Bros. Hat Shop     17868.986719   
                                Sockshop Haight Street    17926.844217   
                                Sunglass Hut                478.638729   

                                                        Airport Service  \
area          categories        name                                      
San Francisco Acai House        Blue Hawaii Açaí Café      19637.364103   
              Accessories Store Coach                        417.205791   
                                Goorin Bros. Hat Shop      17703.968439   
                                Sockshop Haight Street     17768.511901   
                                Sunglass Hut                 250.411778   

                                                        Alternative Healer  \
area          categories        name                                         
San Francisco Acai House        Blue Hawaii Açaí Café          3865.175985   
              Accessories Store Coach                         20661.813705   
                                Goorin Bros. Hat Shop          3270.820319   
                                Sockshop Haight Street         3422.825086   
                                Sunglass Hut                  20478.975122   

                                                        American Restaurant  \
area          categories        name                                          
San Francisco Acai House        Blue Hawaii Açaí Café           4471.327568   
              Accessories Store Coach                          17750.797057   
                                Goorin Bros. Hat Shop

In [134]:
final_df.to_csv("../data/final_sf_only.csv")